### Age-structured SIR and SEIR model for COVID-19 epidemic in India

Fitting Beta, for a new data.

---

* Beta for SIR model

In [ ]:
%matplotlib inline
import numpy as np
import pyross
import pandas as pd
import matplotlib.pyplot as plt      # optional for notebook visualization.
from scipy.io import loadmat         # for loading model's Matlab file.
from scipy import optimize


In [ ]:
M=16  # number of age groups

# load age structure data
my_data = np.genfromtxt('data/age_structures/India-2019.csv', delimiter=',', skip_header=1)
aM, aF = my_data[:, 1], my_data[:, 2]

# set age groups
Ni=aM+aF;   Ni=Ni[0:M];  N=np.sum(Ni)

In [ ]:

# contact matrices
my_data = pd.read_excel('data/contact_matrices_152_countries/MUestimates_home_1.xlsx', sheet_name='India',index_col=None)
CH = np.array(my_data)

my_data = pd.read_excel('data/contact_matrices_152_countries/MUestimates_work_1.xlsx', sheet_name='India',index_col=None)
CW = np.array(my_data)

my_data = pd.read_excel('data/contact_matrices_152_countries/MUestimates_school_1.xlsx', sheet_name='India',index_col=None)
CS = np.array(my_data)

my_data = pd.read_excel('data/contact_matrices_152_countries/MUestimates_other_locations_1.xlsx', sheet_name='India',index_col=None)
CO = np.array(my_data)

my_data = pd.read_excel('data/contact_matrices_152_countries/MUestimates_all_locations_1.xlsx', sheet_name='India',index_col=None)
CA = np.array(my_data)

# matrix of total contacts
C=CH+CW+CS+CO

In [ ]:
def findBeta(x):
    beta=x
    gIa   = 1./7            # recovery rate of asymptomatic infectives 
    gIs   = 1./7            # recovery rate of symptomatic infectives 
    alpha = 0.               # fraction of asymptomatic infectives 
    fsa   = 1                # the self-isolation parameter   


    # initial conditions    
    Is_0 = np.zeros((M));  Is_0[6:13]=3;  Is_0[2:6]=1
    Ia_0 = np.zeros((M))
    R_0  = np.zeros((M))
    S_0  = Ni - (Ia_0 + Is_0 + R_0)
   
    # Loading actual data, kindly update the data by subtracting recovered & no of deaths, so far.
    my_data = np.genfromtxt('data/covid-cases/india.txt', delimiter='', skip_header=4)
    day, cases = my_data[:,0], my_data[:,2]
    
    # duration of simulation and data file, 
    #'Tf' can be changed as per the SME's advice. It is the no. of days considered for finding Beta
    Tf=21;  Nf=np.size(cases); filename='this.mat'

    # the contact structure is independent of time 
    def contactMatrix(t):
        return C

    # intantiate model
    parameters = {'alpha':alpha,'beta':beta, 'gIa':gIa,'gIs':gIs,'fsa':fsa}
    model = pyross.models.SIR(parameters, M, Ni)

    
    # run model
    model.simulate(S_0, Ia_0, Is_0, contactMatrix, Tf, Nf, filename)
    
    data=loadmat(filename); t = data['t'][0]; IC  = np.zeros((Nf))
    for i in range(M):
        IC += data['X'][:,2*M+i] 
    

    error = np.sum(( cases-IC)**2)
    return error



In [ ]:
#solution
beta0  = 0.0


sol = optimize.root( findBeta, beta0)
sol.x

---

* Beta for SEIR model

In [ ]:
def findBetaSEIR(x):
    beta=x
    gIa   = 1./7            # recovery rate of asymptomatic infectives 
    gIs   = 1./7            # recovery rate of symptomatic infectives 
    alpha = 0.               # fraction of asymptomatic infectives 
    gE    = 1./4
    fsa   = 1                # the self-isolation parameter   

    # initial conditions    
    Is_0 = np.zeros((M));  Is_0[6:13]=3;  Is_0[2:6]=1

    E_0 = np.zeros((M))
    Ia_0 = np.zeros((M))
    R_0  = np.zeros((M))
    S_0  = Ni - (Ia_0 + Is_0 + R_0)
    
    # Loading actual data, kindly update the data by subtracting recovered & no of deaths, so far.
    my_data = np.genfromtxt('data/covid-cases/india.txt', delimiter='', skip_header=4)
    day, cases = my_data[:,0], my_data[:,2]

    # duration of simulation and data file, 
    #'Tf' can be changed as per the SME's advice. It is the no. of days considered for finding Beta
    Tf=21;  Nf=np.size(cases); filename='this.mat'

    # the contact structure is independent of time 
    def contactMatrix(t):
        return C

    # intantiate model
    parameters = {'alpha':alpha,'beta':beta, 'gIa':gIa,'gIs':gIs,'gE':gE,'fsa':fsa}
    model = pyross.models.SEIR(parameters, M, Ni)


    # run model
    model.simulate(S_0, E_0, Ia_0, Is_0, contactMatrix, Tf, Nf, filename)
    
    data=loadmat(filename); t = data['t'][0]; IC  = np.zeros((Nf))
    for i in range(M):
        IC += data['X'][:,3*M+i] 
    
    error = np.sum(( cases-IC)**2)
    return error

In [ ]:
#solution
beta0  = 0.0


sol = optimize.root( findBetaSEIR, beta0)
sol.x

---